In [ ]:
!pip3 install --no-cache-dir tensorflow-rocm

In [ ]:
!pip3 install tensorflow

In [ ]:
!pip3 install numpy
!pip3 install nltk

In [ ]:
import math
import numpy as np
import string
import json
import os
import sys
import logging
import nltk
import random
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [ ]:
print(device_lib.list_local_devices())

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
chars = ''
maxlen = 60
step = 3

In [ ]:
print(tf.keras.__version__)
print(tf.__version__)

In [ ]:
# Fix seed's
os.environ['PYTHONHASHSEED']=str(66)
tf.random.set_seed(66)
np.random.seed(66)
random.seed(66)

In [ ]:
def prepareData(dataFile, parts):
    f = open(dataFile,)
    data = json.load(f)

    content = list(data[x] for x in data.keys())
    text = ''

    for c in content[:parts]:
        for i in c:
            text += i

    print(f'Corpus length: {len(text)}')

    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    text = ''
    for c in words:
        text += c
        text += ' '
    text = text.strip()

    print(f'Finished to load file')
    return text

In [ ]:
text = prepareData('Carreira/PROJETOS/PESSOAIS/machine_learning/projects/wikipedia-nlp/wikipedia-content-dataset.json', 250)

In [ ]:
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Numero de sequencias:', len(sentences))

chars = sorted(list(set(text)))
print('Caracteres unicos:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vetorizando o texto')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Finalizado vetorização do texto')

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
model = Sequential([
    LSTM(len(chars), return_sequences=True, input_shape=(maxlen, len(chars))),
    LSTM(len(chars), return_sequences=True),
    LSTM(len(chars)),
    Dense(len(chars), activation='relu'),
    Dense(len(chars), activation='softmax')
])
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print("Treino do modelo")
model.fit(x, y, batch_size=128, epochs=75)